In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
data = pd.read_csv('azureml://subscriptions/512a781e-d15a-4734-adca-96ec827531cb/resourcegroups/xwang3306-rg/workspaces/DVA_PROJECT/datastores/workspaceblobstore/paths/UI/2024-10-16_040004_UTC/consolidated_cleaned_neiss_2014_2023.csv')

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
data['Body_Part_Group']=data['Body_Part'].copy()
data.loc[((data['Body_Part']==84) | (data['Body_Part']==0)| (data['Body_Part']==85)), 'Body_Part_Group'] = 0 # all internal
data.loc[((data['Body_Part']==38) | (data['Body_Part']==79)), 'Body_Part_Group'] = 79 # all internal
data.loc[((data['Body_Part']==80) | (data['Body_Part']==30) | (data['Body_Part']==89)), 'Body_Part_Group'] = 30 # all internal
data.loc[((data['Body_Part']==32) | (data['Body_Part']==33)), 'Body_Part_Group'] = 33 # all internal
data.loc[((data['Body_Part']==88) | (data['Body_Part']==76) | (data['Body_Part']==77)), 'Body_Part_Group'] = 76 # all internal
data.loc[((data['Body_Part']==81) | (data['Body_Part']==35)), 'Body_Part_Group'] = 35 # all internal
data.loc[((data['Body_Part']==87)| (data['Body_Part']==94)), 'Body_Part_Group'] = np.nan # all internal
data.loc[((data['Body_Part']==93) | (data['Body_Part']==83)| (data['Body_Part']==37)), 'Body_Part_Group'] = 37 # all internal
data=data[data['Body_Part_Group'].notna()]

In [3]:
data['Body_Part_Group'].value_counts()

75.0    606221
76.0    425448
37.0    401233
79.0    304586
92.0    285292
30.0    238774
35.0    225049
33.0    208119
31.0    197923
82.0    154695
0.0     131460
36.0    127365
34.0    115960
Name: Body_Part_Group, dtype: int64

In [4]:
def remove_after_dx(narrative):
  if isinstance(narrative, str):
    parts = narrative.split("DX", 1)
    if len(parts) > 1:
      return parts[0]
    else:
      return narrative  # No "DX:" found, return the original string
  else:
    return narrative  # Not a string, return as is


data['Narrative'] = data['Narrative'].apply(remove_after_dx)
data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')


In [5]:
core_col=['Age', 'Sex', 'Race','Location','Hispanic', 'Body_Part_Group','Product_1' ,'Alcohol', 'Drug','Narrative']
data_core=data[core_col]
data_core_sample=data_core.sample(frac=0.1).reset_index(drop=True)

In [6]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


corpus = data_core_sample['Narrative'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative'] = corpus.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')

# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(data_core_sample['Processed_Narrative'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with your existing data
data_ready = pd.concat([data_core_sample, tfidf_df], axis=1)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [7]:
# Separate features (X) and target variable (y)
X = data_ready.drop(['Body_Part_Group','Narrative','Processed_Narrative'], axis=1)  # ,'Processed_Narrative'] Replace 'target_variable' with your target column name
y = data_ready['Body_Part_Group']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

# Combine scaled numerical features with categorical features
X_train_final = X_train_scaled_df.copy()
X_test_final = X_test_scaled_df.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
   # 'Lasso': (Lasso(), {'lasso__alpha': [0.1, 1.0]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1.0], 'svm__kernel': ['linear', 'rbf']}),
    'XGBoost': (XGBClassifier(objective='multi:softmax', num_class=len(le.classes_)),
               {'xgboost__learning_rate': [0.1, 0.01], 'xgboost__max_depth': [3, 5]})
}

# Create an ensemble of models
ensemble_predictions = []
for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy')
    grid_search.fit(X_train_final, y_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions on the test set
    y_pred = best_model.predict(X_test_final)
    ensemble_predictions.append(y_pred)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy:", accuracy)
    print(classification_report(y_test, y_pred))

# Implement your ensemble method (e.g., voting)
# For example, you can use a simple majority vote:
from scipy.stats import mode

ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
print("Ensemble Accuracy:", ensemble_accuracy)
print(classification_report(y_test, ensemble_pred_final))

Training KNN...
KNN Accuracy: 0.6171997136303201
              precision    recall  f1-score   support

           0       0.44      0.61      0.51      2711
           1       0.52      0.54      0.53      4865
           2       0.42      0.44      0.43      4048
           3       0.49      0.52      0.51      4273
           4       0.70      0.56      0.62      2320
           5       0.64      0.56      0.60      4525
           6       0.58      0.44      0.50      2548
           7       0.76      0.78      0.77      7918
           8       0.61      0.71      0.65     11943
           9       0.63      0.55      0.58      8393
          10       0.58      0.54      0.56      6123
          11       0.71      0.67      0.69      3079
          12       0.85      0.79      0.82      5697

    accuracy                           0.62     68443
   macro avg       0.61      0.59      0.60     68443
weighted avg       0.62      0.62      0.62     68443

Training Random_Forest...
Rand

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [8]:
xxx

NameError: name 'xxx' is not defined

In [ ]:
# Separate features (X) and target variable (y)
X = data_ready.drop(['Body_Part_Group','Narrative','Processed_Narrative'], axis=1)  # Replace 'target_variable' with your target column name
y = data_ready['Body_Part_Group']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the XGBoost classifier
model = XGBClassifier(objective='multi:softmax', num_class=len(le.classes_),
                     # Add other hyperparameters as needed (e.g., learning_rate, max_depth)
                     )

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report for detailed metrics
print(classification_report(y_test, y_pred))

In [ ]:
with open('xgb_toy.pkl', 'rb') as f:  # Replace 'your_model_file.pkl' with the actual file name
    loaded_model = pickle.load(f)

In [ ]:
def Predict_Body_parts_Prob(Age, Sex, Race,Location,Hispanic, Product_1 ,Alcohol, Drug):
    X = np.column_stack([Age, Sex, Race,Location,Hispanic, Product_1 ,Alcohol, Drug])
    
    return loaded_model.predict_proba(X)

In [ ]:
Predict_Body_parts_Prob(20,1,0,1,0,1,0,0)

In [ ]:
y_pred_proba = loaded_model.predict_proba(X_test)
y_pred_proba

In [ ]:
import pickle

# Save the model to a file
pickle.dump(model, open("xgb_toy.pkl", "wb"))

In [ ]:
class_labels = le.classes_

# Print the class labels
print("Class Labels:", class_labels)

Word Cloud

In [ ]:
part=[75,76,79,92,31,37,35,82,83,30,36,33,34]  

In [ ]:
for i in part:
    data_filtered=data[data['Body_Part']==i]
    # Assuming you want to create a word cloud from the 'Narrative' column
    text = " ".join(review for review in data_filtered.Narrative.astype(str))

    # Generate the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    # Display the generated image:
    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:

for i in range(2014,2024):
    data_filtered=data[data['Year']==i]
    # Assuming you want to create a word cloud from the 'Narrative' column
    text = " ".join(review for review in data_filtered.Narrative.astype(str))

    # Generate the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    # Display the generated image:
    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
# Assuming you want to create a word cloud from the 'Narrative' column
text = " ".join(review for review in data_filtered.Narrative.astype(str))

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Display the generated image:
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()